# VACC

This notebook ...

## Preparations

In [3]:
# import relevant modules
# define corpus name

## Data Preprocessing

Run the following code to establish the required folder structure. Request the data from Ingo Siegert and place the transcripts in "Fertige_Transkripte" (i.e., place subfolders for each participant there) and the speaker lists in "Fertige_Utterances" (i.e., place subfolders for each participant there).

Dummy folders/files to be able to execute the code?

In [4]:
# establish folder structure
# create csv file
# merge same-speaker turns
# tokenise and lemmatise data (add note that POS-tagging was also performed, but as explained in the thesis this output was not used, hence disregarded here)
# insert note that manual unmerging of turns was performed 
# calculating lemma n-grams
# add note that code for quantitative analysis is in separate notebook due to a different programming language being used (R)

## Persistence Tagging for Qualitative Analysis

In [ ]:
# tagging (only for allo-persistence)
# visualising (change module name)
# inspect frequent persistent n-grams